# HMM GMM

We use 3-state mono-phone HMMs to construct this recognizer. The emission probability of every state is modeled by a GMM. Say we have F mono-phones (F = 39 in our English lexicon), and a G-mixture GMM for each mono-phone state. Thus the GMM-HMM has 3FG mixture components in total. Compared to GMM-UBM, these mixtures are better separated in the phoneme space.

MFCC:  MFCCs  are  extracted  from  16kHz  utterance  with  40 filter-banks distributed between 0 and 8kHz. Static 19-dimensional coefficients plus energy and their delta and delta-delta form a 60-dimensional vector. CMVN is applied per utterance.

Hmm, deepspeech ma 26 MFCC features i zdaje się, że usuwa co drugi, ale to w związku z jakąś sztuczką uczenia RNN. Mają tak: features = mfcc(audio, samplerate=fs, numcep=numcep) gdzie mfcc jest z psf jak u nas xD a numcep=26 domyślnie, a fs=16000 domyślnie

Hmm, inny papier: In our HMM-based method, a phoneme recognizer is firsttrained with 3-state, GMM-based, mono-phone HMMs. Thisrecognizer is the same as in speech recognition. LetFbe thetotal number of mono-phones (i.e. 39),S=3Fbe the numberof all states,Gthe number of Gaussian components per state,andC=SGthe number of all individual Gaussians, and let(s, g)denote Gaussian componentgin states.
3-state - what are you?

Given a transcription, a graph of HMM is composed.

The  Viterbi  and  forward-backward  (FB)  algorithms  are  twomeans to align frames to states and mixtures.

Speaker adaption is the same with Eq. 4 except mixtures here are phonetic dependent.

During the test phase,  the Viterbi-based log-likelihood ratio is expressed as:
sum_t log P(x_t | model_user,qt) - log P(x_t | model_ubm,qt)

HMM: To generate the alignment for the HMM-based modeling,we use MFCCs to train the HMM. 39 mono-phones plus a silencemodel are used,  each of which contains 3 states.  To model thecomplexity of silence, a GMM with 16 mixtures is used for every silence state, while other states are all modeled by 8 Gaussians,resulting 984 Gaussians in total. This HMM is further extended toa triphone system and remains 2142 senones.  The transcriptionsfor  DNN  training  is  generated  by  the  senone  alignment.   OnlyMFCCs are used for HMM training and alignment.

GMM-HMM and i-vector/HMM: The GMM of every state is re-estimated using the HMM alignments and different speaker fea-tures.   The total number of mixtures in our model is 984.   Thedimension of i-vector is again set to 600.  Viterbi and FB align-ments are both investigated

* 39 x 3 states

## MAP - Książka Rabinera:

MAP adaptation of speaker-independent model to a speaker of parameters is obtained by solving 

$$\frac{d}{d\lambda} P(\lambda | O) = 0$$

where

$$P(\lambda | O) = \frac{P(O | \lambda)P(\lambda)}{P(O)}$$

If we have a good prior estimated from a lot of data, then we can use it to get a model for a speaker for which data is scarce.

Conjugate priors for a random vector is defined as the prior distribution for the parameters of the parameters of the probaiblity density function of the random vector, such that the posterior distribution $P(\lambda | O)$ and the prior distribution $P(\lambda)$ belong to the same distribution family for any sample observations $O$.
For example, it is well known that the conjugate prior for the mean of a Gaussian density is also a Gaussian density.

$$\bar{u_{MAP}} = \frac{n \tau^2}{\sigma^2 + n \tau^2} \bar{o} + \frac{\sigma^2}{\sigma^2 + n \tau^2} \rho$$

where $\bar{o}$ - sample mean of new data, $n$ - number of training observations, $\tau^2$ - variance of prior, $\rho$ - mean of prior distribution

How do we estimate $\rho$ and $\tau^2$ of prior? From a collection of speaker-dependent models (and $c_m$ is a weight of a model) or from a speaker-independent model with mixture of distributions in each state ($c_m$ is a weight of a mixture component)

$\rho = \sum_{m=1}^M c_m \rho_m$

$\tau^2 = \sum_{m=1}^M c_m (\rho_m - \rho)$

System do rozpoznawania słowa:

po jednym modelu na słowo, rozpoznając matchujemy z każdym po kolei i bierzemy max prob
model left-to-right zamiast ergodic :OOOO

https://web.stanford.edu/class/cs224s/lectures/

Hmm, Viterbi learning, hard assignment: Bahm ~ o3 
EM learning, soft assignment: Bahm ~ 0*o1 + 0.15*o2 + 0.5*o3 + 0.05*o

Typical training procedure in LVCSR
Generate a forced alignment with existing model
Viterbi decoding with a very constrained prior (the transcript)
Assigns observations to HMM states
Create new observation models from update alignments
Iteratively repeat the above steps, occasionally introducing a more complex observation model or adding more difficult training examples

Viterbi Beam search

Applying FB to speech: Caveats
Network structure of HMM is always created by hand
no algorithm for double-induction of optimal structure and probabilities has been able to beat simple hand-built structures.
Always Bakisnetwork = links go forward in time
Subcase of Bakisnet: beads-on-string net

Assume that the features in the feature vector are uncorrelated
This isn’t true for FFT features, but is true for MFCC features, as we will see later


Initialization: “Flat start”Transition probabilities:set to zero any that you want to be “structurally zero”Set the rest to identical valuesLikelihoods:initialize μand sof each state to global mean and variance of all training data

Embedded TrainingGiven: phoneset, lexicon, transcribed wavefilesBuild a whole sentence HMM for each sentenceInitialize A probsto 0.5, or to zeroInitialize B probsto global mean and varianceRun multiple iterations of Baum WelchDuring each iteration, we compute forward and backward probabilitiesUse them to re-estimate A and BRun Baum-Welch until convergence

Viterbi training rather than Baum-Welch training. Computing the “Viterbi path” over the training data is called “forced alignment”
Because we know which word string to assign to each observation sequence.
We just don’t know the state sequence.
So we use aijto constrain the path to go through the correct words

Modeling phonetic context
The strongest factor affecting phonetic variability is the neighboring phone
Idea: have phone models which are specific to context
Instead of Context-Independent (CI) phonesWe’ll have Context-Dependent (CD) phones


Speech signal is not constantslope of formants, change from stop burst to releaseSo in addition to the cepstralfeaturesNeed to model changes in the cepstralfeatures over time.“delta features”“double delta” (acceleration) features

Typical MFCC featuresWindow size: 25msWindow shift: 10msPre-emphasis coefficient: 0.97MFCC:12 MFCC (mel frequency cepstral coefficients)1 energy feature12 delta MFCC features 12 double-delta MFCC features1 delta energy feature1 double-delta energy featureTotal 39-dimensional features


Acoustic Model AdaptationShift the means and variances of Gaussians to better match the input feature distributionMaximum Likelihood Linear Regression (MLLR)Maximum A Posteriori (MAP) AdaptationFor both speaker adaptation and environment adaptation

MAP Adaptation: u_new = N / (N + a) u_estimated + a / (N + a) u_old, where a is some weight of old

DUŻO BYŁO W lec5

SIL is a phoneme to a recognizerAlways inserted at start and end of utteranceCorrupting silence with bad forced alignments can break recognizer training (silence eats everything)

## Hmm, no dobra, plan działania:

Preprocessing:
wytnij ciszę z dźwięków, voice activity detection 
Wylicz MFCC, 20 stanów + d + dd 

Training:
Dla każdego sentence_id (jest ich 10):
    Utwórz HMMGMM z całego zbioru treningowego sequence_id:
        HMM with 3 states and 8 Gaussian components for each of 39 mono-phones were used for the alignment (resulting in to-tal number of 936 Gaussian components) ?? 
        albo unique(znaki) * 3 stany
        albo znaki * 3 stany trójkątna! (Bakis)
        
Enrollment:
Dla każdego gspeaker_id:
    Dla każdego sentence_id:
        Weź dane (gspeaker_id, sequence_id)
        Naucz HMM taki jak wyżej tylko na nich
        Dokonaj MAP adaptacji
        
Trial:
likelihood HMM_{podany speaker, podane zdanie} - likelihood HMM_{UBM, podane zdanie} ? 

In [1]:
%load_ext autoreload

In [234]:
%autoreload 2

import concurrent.futures as cf
import functools as ft
import itertools as it
import json
import math
import operator as op
import os

from IPython.display import display
from ipywidgets import interact, interact_manual, widgets
from hmmlearn import hmm
from hmmlearn import base as hmmbase
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import interpolate, misc, optimize, spatial, stats
from sklearn import metrics, mixture, cluster, utils

from paprotka.dataset import reddots
from paprotka.feature import cepstral

In [235]:
%autoreload 0

In [4]:
import warnings
warnings.filterwarnings('once')

## Load

In [206]:
root = reddots.get_root()
load_pcm = ft.partial(reddots.load_pcm, root)
load_mfcc = ft.partial(reddots.load_npy, root, 'wac2_mfcc13_ddd')

In [7]:
enrollments_1 = reddots.load_enrollments(root + '/ndx/f_part_01.trn', root + '/ndx/m_part_01.trn')
print('Enrollments', enrollments_1.dtypes, sep='\n')

trials_1 = reddots.load_trials(root + '/ndx/f_part_01.ndx', root + '/ndx/m_part_01.ndx')
print('Trials', trials_1.dtypes, sep='\n')

Enrollments
is_male                  bool
pcm_path               object
sentence_id             int16
speaker_id              int16
timestamp      datetime64[ns]
dtype: object
Trials
correct_sentence                  bool
expected_is_male                  bool
expected_sentence_id             int16
expected_speaker_id              int16
pcm_path                        object
target_person                     bool
trial_is_male                     bool
trial_sentence_id                int16
trial_speaker_id                 int16
trial_timestamp         datetime64[ns]
dtype: object


In [8]:
# script = reddots.load_script(root + '/infos/script.txt')
script = pd.read_csv(root + '/infos/phones.csv', sep=';')
print(script.loc[script['sentence_id'] == 31])

enrollments_1 = enrollments_1.merge(script, how='left', on='sentence_id')
trials_1 = trials_1.merge(script, how='left', left_on='expected_sentence_id', right_on='sentence_id')
del trials_1['sentence_id']
trials_1.rename(columns={'content': 'expected_content', 'phones': 'expected_phones'}, inplace=True)
trials_1 = trials_1.merge(script, how='left', left_on='trial_sentence_id', right_on='sentence_id')
del trials_1['sentence_id']
trials_1.rename(columns={'content': 'trial_content', 'phones': 'trial_phones'}, inplace=True)

      sentence_id                  content  \
1609           31  My voice is my password   

                                                 phones  
1609  M AY1 _ V OY1 S _ IH1 Z _ M AY1 _ P AE1 S W ER2 D  


## Train

In [208]:
example = enrollments_1[enrollments_1.sentence_id == 31]
example_features = [load_mfcc(path) for path in example['pcm_path']]
example_lengths = [mfcc.shape[0] for mfcc in example_features] 
example_features_stack = np.vstack(example_features)

In [184]:
def wrap_with_silence(phones):
    return ['_'] + phones + ['_']

def remove_stress_combinations(phones):
    def remove_single_phone(phone):
        if phone[-1].isdigit():
            return phone[:-1]
        else:
            return phone
        
    return [remove_single_phone(phone) for phone in phones]

example_phones = example.phones.loc[0].split(' ')
example_phones_cleaned = wrap_with_silence(remove_stress_combinations(example_phones))
# example_phones_cleaned = remove_stress_combinations(example_phones)
example_phones_unique = np.unique(example_phones_cleaned)

print(example_phones)
print(example_phones_cleaned)
print(example_phones_unique)

['M', 'AY1', '_', 'V', 'OY1', 'S', '_', 'IH1', 'Z', '_', 'M', 'AY1', '_', 'P', 'AE1', 'S', 'W', 'ER2', 'D']
['_', 'M', 'AY', '_', 'V', 'OY', 'S', '_', 'IH', 'Z', '_', 'M', 'AY', '_', 'P', 'AE', 'S', 'W', 'ER', 'D', '_']
['AE' 'AY' 'D' 'ER' 'IH' 'M' 'OY' 'P' 'S' 'V' 'W' 'Z' '_']


In [209]:
## unique
n_components = 3 * example_phones_unique.size
example_classifier_unique = hmm.GMMHMM(n_components=n_components, n_mix=8)
example_classifier_unique.fit(example_features_stack, example_lengths);

In [210]:
bg_model = mixture.GaussianMixture(n_components=8, covariance_type='diag')
bg_model.fit(example_features_stack)

GaussianMixture(covariance_type='diag', init_params='kmeans', max_iter=100,
        means_init=None, n_components=8, n_init=1, precisions_init=None,
        random_state=None, reg_covar=1e-06, tol=0.001, verbose=0,
        verbose_interval=10, warm_start=False, weights_init=None)

In [157]:
def make_matrix_beads(size):
    initial = np.zeros(size)
    initial[0] = 1.0
    transitions = np.eye(size)
    transitions[:, 1:] += np.eye(size, size - 1)
#     transitions[-1, -2] = 1
    transitions /= transitions.sum(axis=1)[:, np.newaxis]
    return initial, transitions

def make_matrix_triangular_beads(size, phones, phone_size=3, silence_size=1):
    initial = np.zeros(size)
    initial[0] = 1.0
    
    next_ix = 0
    starts = np.zeros(size)
    for phone in phones:
        starts[next_ix] = 1
        if phone == '_':
            next_ix += silence_size
        else:
            next_ix += phone_size
    ends = starts[1:]
    
    transitions = np.eye(size)
    transitions[:, 1:] += np.eye(size, size - 1)
#     transitions[-1, -2] = 1

    for end_ix in np.where(ends)[0]:
        transitions[end_ix, end_ix+1:] = starts[end_ix+1:]

    transitions /= transitions.sum(axis=1)[:, np.newaxis]
    return initial, transitions

make_matrix_beads(7)
print(make_matrix_triangular_beads(9, ['_', 'A', 'B', '_', '_']))

(array([ 1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]), array([[ 0.2       ,  0.2       ,  0.        ,  0.        ,  0.2       ,
         0.        ,  0.        ,  0.2       ,  0.2       ],
       [ 0.        ,  0.5       ,  0.5       ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.5       ,  0.5       ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.25      ,  0.25      ,
         0.        ,  0.        ,  0.25      ,  0.25      ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.5       ,
         0.5       ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.5       ,  0.5       ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.33333333,  0.33333333,  0.33333333],
       

In [216]:
print(bg_model.weights_)
print(bg_model.means_[0:2])
print(bg_model.covariances_[0:2])

[ 0.17406918  0.07971898  0.06388068  0.2542175   0.1666166   0.06764632
  0.06812612  0.12572463]
[[  1.12804076e+01  -1.43035635e+01  -1.33024575e-01  -9.46662018e-01
    1.42936349e+00   1.02261308e+00  -3.38650901e+00  -6.84189738e-01
   -2.51636292e+00   6.76333596e-01  -2.08126090e+00   1.04897852e+00
   -1.28166446e+00  -2.42064069e-02  -8.01072658e-02   6.08988313e-02
    9.19454598e-03   4.97571247e-02   4.80897194e-02   4.50676825e-02
    8.28164009e-03  -6.21901271e-02  -6.68429365e-02   6.42958377e-03
    1.35040588e-02  -1.78331209e-02   9.72259057e-03   2.09559059e-02
    4.71504369e-03   1.80189763e-02  -4.71426935e-03  -6.25000137e-03
   -8.06028949e-03  -2.43505710e-02  -2.17094405e-02  -7.91273859e-03
   -1.62002664e-03  -8.51477843e-03  -1.63135388e-02]
 [  1.82575007e+01  -3.39841582e+01   1.55370353e+01  -7.54255772e+00
   -3.62655996e+00   1.59190706e+00  -1.33740901e+01   7.06393712e-01
   -2.76356502e+00  -9.35160035e-01  -1.61003912e+00  -1.58955059e+00
   -1.7

In [230]:
## beads
# params=subset of 'stmcw'
def calculate_n_components(phones, phone_n=3, silence_n=1):
    n_components = phone_n * len(phones)
    
    def silence_indices(phone):
        return [True] * silence_n if phone == '_' else [False] * phone_n
    silences = np.array([v for phone in phones for v in silence_indices(phone) ], dtype=np.bool)
    
    if phone_n != silence_n:
        difference = silence_n - phone_n
        n_components += silences.sum() * difference
    return n_components, silences

def make_classifier_beads(phones, bg_model):
    n_components, silences = calculate_n_components(phones)
#     initial, transitions = make_matrix_triangular_beads(n_components, phones)
    initial, transitions = make_matrix_beads(n_components)

    classifier = hmm.GMMHMM(
        n_components=n_components, n_mix=8, covariance_type='diag', init_params='', params='tmcw'
    )

    classifier.startprob_ = initial
    classifier.transmat_ = transitions
    classifier.weights_ = np.tile(bg_model.weights_, (n_components, 1))
    means = np.tile(bg_model.means_, (n_components, 1, 1))
    means += np.random.normal(scale=0.1, size=means.shape)
    classifier.means_ = means
    covars = np.tile(bg_model.covariances_, (n_components, 1, 1))
#     covars += np.abs(np.random.normal(scale=0.01, size=covars.shape))
    classifier.covars_ = covars 
#     classifier.covars_[silences] *= 5
    
    return classifier

example_classifier_beads = make_classifier_beads(example_phones_cleaned, bg_model)
example_classifier_beads.fit(example_features_stack, example_lengths)

GMMHMM(algorithm='viterbi', covariance_type='diag',
    covars_prior=array([[[-1.5, -1.5, ..., -1.5, -1.5],
        [-1.5, -1.5, ..., -1.5, -1.5],
        ...,
        [-1.5, -1.5, ..., -1.5, -1.5],
        [-1.5, -1.5, ..., -1.5, -1.5]],

       [[-1.5, -1.5, ..., -1.5, -1.5],
        [-1.5, -1.5, ..., -1.5, -1.5],
        ...,
        [-1.5, -1.5, .....-1.5],
        ...,
        [-1.5, -1.5, ..., -1.5, -1.5],
        [-1.5, -1.5, ..., -1.5, -1.5]]]),
    covars_weight=array([[[ 0.,  0., ...,  0.,  0.],
        [ 0.,  0., ...,  0.,  0.],
        ...,
        [ 0.,  0., ...,  0.,  0.],
        [ 0.,  0., ...,  0.,  0.]],

       [[ 0.,  0., ...,  0.,  0.],
        [ 0.,  0., ...,  0.,  0.],
        ...,
        [ 0.,  0., ...,  0.,  0.],
        [ 0.....,  0.,  0.],
        ...,
        [ 0.,  0., ...,  0.,  0.],
        [ 0.,  0., ...,  0.,  0.]]]),
    init_params='',
    means_prior=array([[[ 0.,  0., ...,  0.,  0.],
        [ 0.,  0., ...,  0.,  0.],
        ...,
        [ 0.,  0.

In [231]:
for i in (0, 50, 100):
    print(example_classifier_beads.decode(example_features[i], algorithm='viterbi'))
#     print(example_classifier_beads.decode(example_features[i], algorithm='map'))

(-52967.77978585746, array([ 0,  1,  1,  1,  1,  1,  1,  2,  3,  4,  5,  6,  7,  8,  8,  8,  8,
        8,  8,  8,  9, 10, 11, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
       16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16,
    

In [232]:
print('Means')
print(example_classifier_beads.means_[0:4, 0:4, 0:4])
print(example_classifier_beads.means_[-2:, 0:4, 0:4])
print('Covars')
print(example_classifier_beads.covars_[0:4, 0:4, 0:4])
# np.tile(bg_model.means_, (n_components, 1, 1))[0:4, 0:4, 0:4]
# example_classifier_beads.means_.shape
print(example_classifier_beads.transmat_.diagonal())
print(example_phones_cleaned)

Means
[[[ 16.25947214   0.32716725  18.09623335   3.99491784]
  [ 16.25947213   0.32716731  18.0962334    3.99491783]
  [ 16.25947214   0.32716722  18.09623333   3.99491784]
  [ 16.25947214   0.32716725  18.09623335   3.99491784]]

 [[ 12.82098929 -14.92699264  24.71795361   6.32869305]
  [ 12.82098929 -14.92699264  24.71795361   6.32869305]
  [ 12.82098929 -14.92699264  24.71795361   6.32869305]
  [ 12.82098929 -14.92699264  24.71795361   6.32869305]]

 [[ 12.50236988 -22.27513711  14.89559182  -2.48796384]
  [ 12.50236988 -22.27513711  14.89559182  -2.48796384]
  [ 12.50236988 -22.27513711  14.89559182  -2.48796384]
  [ 12.50236988 -22.27513711  14.89559182  -2.48796384]]

 [[ 12.63585093 -28.44920449   9.11174904  -9.34496307]
  [ 12.63585093 -28.44920449   9.11174904  -9.34496307]
  [ 12.63585093 -28.44920449   9.11174904  -9.34496307]
  [ 12.63585093 -28.44920449   9.11174904  -9.34496307]]]
[[[ 12.96333628 -21.60978125  17.38953096  -6.44232739]
  [ 12.9633362  -21.60980235  17.3

In [ ]:
def perform_training(classifier, enrollments):
    labels = enrollments[['is_male', 'speaker_id', 'sentence_id']].values
    features = [load_mfcc(path) for path in enrollments_1['pcm_path']]
    classifier.fit(features, labels)

## Enroll

In [54]:
def make_speaker_classifier(inspeaker_classifier):
    n_components = 3 * len(phones)
    initial, transitions = make_matrix_beads(n_components)

    classifier = hmm.GMMHMM(
        n_components=n_components, n_mix=8, covariance_type='diag', init_params='mcw', params='tmcw'
    )

    classifier.startprob_ = initial
    classifier.transmat_ = transitions

GMMHMM(algorithm='viterbi', covariance_type='diag', covars_prior=0.01,
    init_params='stmcw', n_components=39, n_iter=10, n_mix=8,
    params='stmcw', random_state=None, startprob_prior=1.0, tol=0.01,
    transmat_prior=1.0, verbose=False)

In [ ]:
def perform_enrollments(classifier, enrollments):
    labels = enrollments[['is_male', 'speaker_id', 'sentence_id']].values
    features = [load_mfcc(path) for path in enrollments_1['pcm_path']]
    classifier.fit(features, labels)
    

## Trial

In [ ]:
def save_results(label, results):
    path = os.path.join(root, 'result', label)
    with open(path) as opened:
        pickle.dump(results, opened)
        
def load_results(label):
    path = os.path.join(root, 'result', label)
    with open(path) as opened:
        return pickle.load(opened)
    
def perform_trial(classifier, path):
    features = load_mfcc(path)
    return classifier.predict_single_proba(features)

def perform_trials(classifier, trials):
    paths = trials['pcm_path'].unique()
    results = {}
    for path in paths:
        results[path] = perform_trial(classifier, path)
    return results

In [10]:
per_phone_weights = {}

per_phone_weights.get('A', np.eye(3))

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [ ]:
class Bakis:
    def __init__(self, phones, per_phone_weights, ignore_stress=True, wrap_silence=True):
        self.phone_seq = phones.split(' ')
        if wrap_silence:
            self.phone_seq = ['_'] + self.phone_seq + ['_']
            
        state_transitions = 

In [51]:
enrollments_1.sentence_id.unique()

array([31, 32, 33, 34, 35, 36, 37, 38, 39, 40])

In [52]:
sample_enrollments = enrollments_1.loc[enrollments_1.sentence_id == 31]

In [ ]:
class HmmGmmVerifier:
    def __init__(self):
        self.patterns = None
        self.labels = None

    def fit(self, features, labels):
        self.patterns = features
        self.labels = labels
        self.unique_labels = np.unique(labels)
        
    def enroll(self, ):
        pass
    
    def trial(self, ):
        pass

    def predict(self, features, metric=spatial.distance.cosine):
        sequence_label_proba = self.predict_proba(features, metric)
        max_proba_index = sequence_label_proba.argmax(axis=1)
        return self.unique_labels[max_proba_index]
    
    def predict_proba(self, features, metric=spatial.distance.cosine):
        sequence_n = len(features) 
        pattern_n = len(self.patterns)
        
        sequence_label_proba = np.zeros((sequence_n, pattern_n), dtype=self.labels.dtype)
        for i, sequence in enumerate(features):
            sequence_label_proba[i, :] = self.predict_single_proba(sequence, metric)
            
        return sequence_label_proba
    
    def predict_single_proba(self, sequence, metric=spatial.distance.cosine):
        pattern_dists = np.zeros(len(self.patterns), dtype=np.float64)
        for i, pattern in enumerate(self.patterns):
            distance, _ = fastdtw.fastdtw(pattern, sequence, dist=metric)
            pattern_dists[i] = distance
            
        pattern_proba = np.exp(-pattern_dists)
        
        label_proba = np.zeros(len(self.unique_labels), dtype=np.float64)
        all_dim = tuple(range(1, self.labels.ndim))
        for i, label in enumerate(self.unique_labels):
            relevant = (self.labels == label).all(axis=all_dim)
            total_proba = pattern_proba[relevant].sum()
            label_proba[i] = total_proba
        
        return label_proba / label_proba.sum()

classifier = markov.HMMGMMClassifier()
classifier.fit(e_features, e_labels, n_components=10, n_mix=2)
return classifier.predict(t_features)

In [5]:
display(enrollments_1.groupby(['is_male', 'speaker_id']).size())

is_male  speaker_id
False    2             30
         4             30
         5             30
         6             30
         8             24
         12            30
True     1             30
         2             30
         4             30
         5             30
         6             30
         7             30
         8              6
         9             30
         13            30
         14            24
         15            30
         16            30
         17            30
         18            30
         19            30
         20            30
         21            24
         22            30
         23            30
         26            30
         28            30
         29            30
         32            30
         38            24
         40            30
         41            30
         43            30
         47            30
         48             6
         51            30
         52             6
         53       